In [1]:
import re,math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy.stats import pearsonr
from scipy.optimize import leastsq


data1 = pd.read_csv(r'细胞间相似性（皮尔森）.csv',engine='python')
# 细胞系-细胞系
high_data = data1.set_index(data1.loc[:,"Unnamed: 0"]).drop(["Unnamed: 0"], axis=1)
# 药物对-细胞系
data2 = pd.read_csv('label_gai.csv')
# 选取细胞系名称
columns = list(data2.columns[1:])
# 删去对应行后的字典
dict_high_lines = {}
count = 0
for i in high_data.index:
    y = list(high_data.loc[:,i])
    y.pop(count)
    count += 1  
    dict_high_lines[i] = y
# 分离x y数据构造字典
dict_columns = {}
for i in columns: 
    
    # b
    y = data2.loc[:,i]
    
    # A
    temp = []
    for j in columns:
        if j != i:
            temp.append(j)
    m = data2.loc[:, temp[0]]
    tp = [m]
    for j in temp[1:]:
        tp.append(data2.loc[:, j])
    x = pd.DataFrame(tp).T
    
    tp = []
    for j in columns:
        if i != j:
            tp.append(j)
    df = pd.DataFrame(dict_high_lines[i], index=tp)
    df.sort_values(by=0 , inplace=True, ascending=True)
    max_l = df.index[-13:]
    tp = []
    for j in max_l:
        tp.append(data2.loc[:, j])
    max_5x = pd.DataFrame(tp).T
    
    # 存储
    data = {
        "x": x,
        "y": y,
        "max_relevance": max_l,
        "max_5x": max_5x
    }
    dict_columns[i] = data
# 删去对应行后的字典
dict_high_lines = {}
count = 0
for i in high_data.index:
    y = list(high_data.loc[:,i])
    y.pop(count)
    count += 1
    dict_high_lines[i] = y

    tp = []
    for j in columns:
        if i != j:
            tp.append(j)
    df = pd.DataFrame(dict_high_lines[i], index=tp)
    df.sort_values(by=0 , inplace=True, ascending=True)
    
    max_l = df.index[-5:] 
    tp = []
    for j in max_l:
        tp.append(high_data.loc[j, :])
    max_5y = list(pd.DataFrame(tp).T.loc[[i]].values)[0]
    
    data = {
        "max_relevance": max_l,
        "y": y,
        "high_5": max_5y
    }
    dict_high_lines[i] = data

cell_dict_columns=dict_columns
cell_dict_high_lines=dict_high_lines

In [2]:
data1 = pd.read_excel(r'pcbi.1006752.s002.xlsx', sheet_name="Sheet1",index_col='Drug').drop('SMILES',axis=1)
data2 = pd.read_excel(r'Label_synergy4000.xlsx')
# 合并drugA和drugB为一列作为索引
temp1 = []
index_l = []
for index,row in data2.iterrows():
    ct = np.array(data1.loc[row["DrugA"],:]) + np.array(data1.loc[row["DrugB"],:])
    tp = []
    for i in ct:
        if i >= 1:
            tp.append(1)
        else:
            tp.append(0)
    temp1.append(tp)
    index_l.append((row["DrugA"],row["DrugB"]))
# 两组数据的tanimoto计算
def tanimoto(p,q):
    tep1=0
    tep2=0
    lenthp=int(len(p))
    for i in range(lenthp):
        a=p[i]        
        b=q[i]
        if (a==1)|(b==1):
            tep1=tep1+1
        if (a==1)&(b==1):
            tep2=tep2+1 
    c=float(tep2 / tep1)        
    return c
# 相互之间做一次tanimoto
temp=[]  
for i in temp1:
    tmp=[]   
    a=np.array(i) #取出一组值
    for j in temp1:
        b=np.array(j)    #取出一组值
        tp=tanimoto(a,b)
        tmp.append(tp)
    temp.append(tmp)
Tanimoto=pd.DataFrame(temp,columns=index_l,index=index_l)
# 获取栏目数组
ttpp = []
for i in list(Tanimoto.columns):
    ttpp.append(str(i)[1:-1])
data2 = pd.DataFrame(data2.iloc[:,2:].T.values, columns=list(ttpp))
columns = data2.columns
# 删去对应行后的字典
dict_lines = {}
count = 0
ct = 0
for i in Tanimoto.index:
    y = list(Tanimoto.iloc[:,ct])
    y.pop(count)
    count += 1
    dict_lines[columns[ct]] = y
    ct += 1
# 分离x y数据构造字典
dict_columns = {}
for i in columns: 
    
    # b
    y = data2.loc[:,i]
    
    # A
    temp = []
    for j in columns:
        if j != i:
            temp.append(j)
    m = data2.loc[:, temp[0]]
    tp = [m]
    for j in temp[1:]:
        tp.append(data2.loc[:, j])
    x = pd.DataFrame(tp).T
    

    tp = []
    for j in columns:
        if i != j:
            tp.append(j)
    df = pd.DataFrame(dict_lines[i], index=tp)
    df.sort_values(by=0 , inplace=True, ascending=True)
    max_l = df.index[-5:]
    tp = []
    for j in max_l:
        tp.append(data2.loc[:, j])
    max_5x = pd.DataFrame(tp).T
    
    # 存储
    data = {
        "x": x,
        "y": y,
        "max_relevance": max_l,
        "max_5x": max_5x
    }
    dict_columns[i] = data
# 删去对应行后的字典
dict_high_lines = {}
count = 0
cct = 0
for i in Tanimoto.index:
    y = list(Tanimoto.iloc[:,cct])
    y.pop(count)
    count += 1
    dict_high_lines[str(i)[1:-1]] = y

    tp = []
    for j in columns:
        if str(i)[1:-1] != j:
            tp.append(j)
    df = pd.DataFrame(dict_high_lines[str(i)[1:-1]], index=tp)
    df.sort_values(by=0 , inplace=True, ascending=True)

    max_l = df.index[-5:] 
    tp = []
    for j in max_l:
        tp.append(Tanimoto.iloc[list(columns).index(j), :])
    max_5y = list(pd.DataFrame(tp).T.iloc[[list(columns).index(str(i)[1:-1])]].values)[0]

    data = {
        "y": y,
        "max_relevance": max_l,
        "high_5": max_5y
    }
    dict_high_lines[str(i)[1:-1]] = data
    cct += 1

drug_dict_columns=dict_columns
drug_dict_high_lines=dict_high_lines

In [3]:
def dictW_C(alpha):
    # 运算构造参数W字典 
    dict_W = {}
    count = 0
    for elem in cell_dict_high_lines:
        r = np.array(cell_dict_high_lines[elem]["high_5"])
        W1= pow((1-r),2)
        W = np.exp(-W1/(2*pow(alpha[count],2)))
        dict_W[elem] = W
        count+=1
    return dict_W

def dictW_D(alpha):
    # 运算构造参数W字典 
    dict_W = {}
    count = 0
    for elem in drug_dict_high_lines:
        r = np.array(drug_dict_high_lines[elem]["high_5"])
        W1= pow((1-r),2)
        W = np.exp(-W1/(2*pow(alpha[count],2)))
        dict_W[elem] = W
        count+=1
    return dict_W

def dictr(dict_W):
    # 均方误差字典 （MSE=
    # r = max(|A×W - b|)
    dict_r = {}
    for elem,w in zip(dict_columns, dict_W):
        A = np.array(dict_columns[elem]["max_5x"].values) # x----->max_5x
        b = np.array(dict_columns[elem]["y"].values)
        W = dict_W[w]
        temp1 = np.dot(A,W) - b
        temp2 = np.square(temp1)
        temp3 = np.mean(temp2)
        dict_r[w] = temp3
    return dict_r

def dirlist(dict_r):
    #计算误差平方和    
    dir_list = []
    for elem in dict_r:
        dir_list.append(np.sqrt(dict_r[elem]))
    return dir_list

In [3]:
import pickle

with open('细胞系alpha.pkl', 'rb') as file:
    cell_dict_alpha = pickle.load(file)

EOFError: Ran out of input

In [2]:
import pickle

with open('药物对alpha.pkl', 'rb') as file:
    drug_dict_alpha = pickle.load(file)

In [4]:
max(drug_dict_alpha)

0.06326530612244897

In [6]:
cell_alp=[0.1 for i in range(35)]
drug_alp=[0.1 for i in range(583)]
cell_alp_copy=cell_alp
drug_alp_copy=drug_alp

In [20]:
cell_dict_W = dictW_C(drug_dict_alpha)
drug_dict_W = dictW_D(cell_dict_alpha)

data1 = pd.read_excel(r'pcbi.1006752.s002.xlsx', sheet_name="Sheet1",index_col='Drug').drop('SMILES',axis=1)
data2 = pd.read_excel(r'Label_synergy4000.xlsx')
data2_columns = data2.columns[2:]

# 结构  {名字： 名字列表， 权重： 权重列表} 顺序对应
dict_molecule_cell = {}
for elem in cell_dict_high_lines:
    dict_molecule_cell[elem] = []
    dict_molecule_cell[elem].append(cell_dict_columns[elem]["max_relevance"][-13:])
    dict_molecule_cell[elem].append(cell_dict_W[elem][-13:])

dict_molecule_drug = {}
for elem in drug_dict_high_lines:
    dict_molecule_drug[elem] = []
    dict_molecule_drug[elem].append(drug_dict_columns[elem]["max_relevance"][-5:])
    dict_molecule_drug[elem].append(drug_dict_W[elem][-5:])

# 拆分索引获取每行对应的drugA和drugB
drugA = []
drugB = []
for i in dict_molecule_drug:
    drugA.append(i.split(", ")[0][1:-1])
    drugB.append(i.split(", ")[1][1:-1])

data22=data2

# 获取得分高的细胞系名称组
cells = []
for i in dict_molecule_cell:
    cells.append(i)
cells.extend(data22.columns[0:2])


# 从原始数据中按列取得分高的细胞系
temp = []
for column,line in data22.iteritems():
    if column in cells:
        temp.append(line)
data222 = pd.DataFrame(temp).T

# 筛选结果合并drugA和drugB
tptt = []
for i,j in zip(data222['DrugA'], data222['DrugB']):
    tptt.append(f"'{i}', '{j}'")
data222 = pd.DataFrame(data222.iloc[:,2:].values,index=tptt,columns=data222.columns[2:])


# 原始数据合并drugA和drugB
tptt = []
for i,j in zip(data2['DrugA'], data2['DrugB']):
    tptt.append(f"'{i}', '{j}'")
data3 = pd.DataFrame(data2.iloc[:,2:].values,index=tptt,columns=data2.columns[2:])
data2 = data3.copy()

# 预测筛选后的药物对与细胞系的协同得分
result_df = []
for i in data222.index:
    tp = []
    for k in data222.columns[0:]:
        molecule = 0
        denominator = 0
        a = dict_molecule_cell[k][0]
        b = dict_molecule_drug[i][0]
        cw = dict_molecule_cell[k][1]
        dw = dict_molecule_drug[i][1]
        for m,ccw in zip(a,cw): # 细胞系   名字  权重
            for n,ddw in zip(b,dw): # 药物对   名字   权重
                # 分子子项 = 细胞系权重 × data2对应名字的值 ×药物对权重
                temp_molecule = ccw*data2[data2.index==n][m].values[0]*ddw
                # 分母子项 = 细胞系权重 × 药物对权重
                #temp_denominator = ccw*ddw
                temp_denominator=1
                molecule += temp_molecule
                denominator += temp_denominator
        result = molecule / denominator
        tp.append(result)
    result_df.append(tp)

sens=pd.DataFrame(np.array(result_df),index = data222.index, columns=data222.columns)

MSE=pow(data222-sens,2).sum().sum()/(sens.shape[0]*sens.shape[1])

MSE

526.5190606520167

In [14]:
dict_molecule_cell

{'A2058_SKIN ': [Index(['RPMI7951_SKIN ', 'COLO320_LARGE_INTESTINE ', 'MSTO211H_PLEURA ',
         'HT29_LARGE_INTESTINE ', 'NCIH1650_LUNG ', 'SW620_LARGE_INTESTINE ',
         'CAOV3_OVARY ', 'A2780_OVARY ', 'MDAMB436_BREAST ', 'HT144_SKIN ',
         'SKMEL30_SKIN ', 'UACC62_SKIN ', 'NCIH460_LUNG '],
        dtype='object'),
  array([0.45370533, 0.51174746, 0.52629189, 0.56598286, 0.60980863])],
 'A2780_OVARY ': [Index(['HT29_LARGE_INTESTINE ', 'A427_LUNG ', 'OVCAR4_OVARY ',
         'MDAMB436_BREAST ', 'SKMEL30_SKIN ', 'NCIH460_LUNG ',
         'SW837_LARGE_INTESTINE ', 'LOVO_LARGE_INTESTINE ', 'CAOV3_OVARY ',
         'SW620_LARGE_INTESTINE ', 'A2058_SKIN ', 'VCAP_PROSTATE ',
         'COLO320_LARGE_INTESTINE '],
        dtype='object'),
  array([0.53409692, 0.53856768, 0.55295517, 0.69884354, 0.84360473])],
 'A375_SKIN ': [Index(['CAOV3_OVARY ', 'HCT116_LARGE_INTESTINE ', 'HT29_LARGE_INTESTINE ',
         'NCIH520_LUNG ', 'LOVO_LARGE_INTESTINE ', 'SKMEL30_SKIN ',
         'ZR751_B

In [8]:
ccw

0.11709416665665782

In [13]:
cw

array([0.01152471, 0.01243092, 0.01388881, 0.0210057 , 0.11709417])

In [12]:
data2

,A2058_SKIN,A2780_OVARY,A375_SKIN,A427_LUNG,CAOV3_OVARY,COLO320_LARGE_INTESTINE,EFM192A_BREAST,ES2_OVARY,HCT116_LARGE_INTESTINE,HT144_SKIN,...,SKMES1_LUNG,SKOV3_OVARY,SW620_LARGE_INTESTINE,SW837_LARGE_INTESTINE,T47D_BREAST,UACC62_SKIN,VCAP_PROSTATE,ZR751_BREAST,DLD1_LARGE_INTESTINE,PA1_OVARY
"'MK-5108', 'SORAFENIB'",-9.510,2.60000,15.200,6.22000,-16.4000,5.360,-7.080,8.050,12.400,5.7900,...,-6.2300,13.500,5.100,5.150,6.630,3.880,-1.480,-8.660,13.000,4.8300
"'VINORELBINE', 'SUNITINIB'",-13.200,-4.03000,11.100,10.50000,-15.6000,16.300,-1.720,8.890,0.686,9.6800,...,2.8200,1.510,13.800,-5.770,-14.400,-10.700,-16.800,-11.800,15.300,-13.1000
"'SUNITINIB', 'MK-8776'",26.400,14.50000,29.500,17.50000,17.7000,10.300,0.830,5.770,19.300,8.4000,...,-12.2000,-7.400,10.500,-4.970,13.100,18.800,12.000,5.950,6.770,18.9000
"'5-FU', 'DINACICLIB'",4.330,-8.16000,-5.420,-7.75000,-14.3000,4.810,0.312,3.410,2.920,-3.4800,...,-3.1400,-13.900,-4.570,-19.600,-2.720,3.680,14.400,-13.600,-3.640,-5.7300
"'SUNITINIB', 'MK-2206'",45.500,22.50000,22.800,18.00000,20.1000,22.700,6.300,24.900,35.400,23.4000,...,24.4000,31.200,18.400,12.900,14.300,34.700,21.200,14.400,26.400,55.4000
"'PACLITAXEL', 'BEZ-235'",-5.010,-3.86000,25.500,4.91000,-15.3000,21.200,-30.800,4.260,-6.130,27.1000,...,27.9000,14.900,8.650,0.875,97.800,12.900,4.880,24.200,17.800,-7.4800
"'MK-2206', 'DINACICLIB'",9.970,1.69000,13.200,7.66000,19.2000,7.130,-9.490,14.700,4.010,6.5200,...,8.3800,-0.318,6.350,-0.448,1.460,12.400,-20.900,8.510,16.100,6.4000
"'MK-4827', 'MK-8776'",76.200,21.60000,38.700,-12.10000,22.4000,13.500,-12.000,-53.000,17.400,21.8000,...,-1.7400,6.090,12.600,-11.500,-14.800,7.910,18.600,7.200,7.200,30.5000
"'VINORELBINE', 'DINACICLIB'",-16.000,-19.20000,-123.000,-4.42000,-27.2000,17.400,-13.600,11.500,-25.100,-0.0348,...,-7.7700,-15.200,-1.810,-12.800,-13.100,13.300,-36.700,7.440,8.100,-24.1000
"'MK-5108', 'MK-8776'",-4.970,11.10000,8.950,6.89000,-1.1600,-48.400,16.800,3.900,8.670,-6.3600,...,-9.6300,20.100,1.530,-9.930,-8.420,4.380,-1.660,-8.310,-8.440,6.2800
